Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [1]:
from collections import Counter
from libs.model import Model
import numpy as np
import time
from libs.mcts import MCTS

In [2]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
WINDOW_LENGTH = 4
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [3]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )

## Montecarlo Evaluation

In [17]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if four_in_a_row(board, p):
            return p
    return 0


def montecarlo_e(board, player):
    montecarlo_samples = 100
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples

## Heuristic evaluation

In [5]:
def evaluate_window(window, player):
    score = 0
    opp_player = -player

    if window.count(player) == 4:
        score += 100
    elif window.count(player) == 3 and window.count(0) == 1:
        score += 5
    elif window.count(player) == 2 and window.count(0) == 2:
        score += 2

    if window.count(opp_player) == 3 and window.count(0) == 1:
        score -= 4

    return score


# used to evaluate a position using simple heuristics:
# a player position "earns" scores if there are disks in the center and/or there are two or more disks are connected
# a player position "lose" scores if the opposing player has two or more disks connected
def score_position(board, player, to_print=False):
    score = 0
    # Score center column
    center_array = [int(i) for i in list(board[NUM_COLUMNS//2, :])]
    center_count = center_array.count(player)
    score += center_count * 3
    
    if to_print:
        print('center_array')
        print(center_array, ' ', score, ' ', center_count)

    # Score Horizontal
    for r in range(NUM_COLUMNS):
        row_array = [int(i) for i in list(board[r, :])]
        for c in range(COLUMN_HEIGHT-3):
            window = row_array[c:c+WINDOW_LENGTH]
            score += evaluate_window(window, player)

    if to_print:
        print('horizontal')
        print(score)

    # Score Vertical
    for c in range(COLUMN_HEIGHT):
        col_array = [int(i) for i in list(board[:, c])]
        for r in range(NUM_COLUMNS-3):
            window = col_array[r:r+WINDOW_LENGTH]
            score += evaluate_window(window, player)

    if to_print:
        print('vertical')
        print(score)

    # Score positive sloped diagonal
    for r in range(NUM_COLUMNS-3):
        for c in range(COLUMN_HEIGHT-3):
            window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, player)

    for r in range(NUM_COLUMNS-3):
        for c in range(COLUMN_HEIGHT-3):
            window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
            score += evaluate_window(window, player)

    if to_print:
        print('diagonal')
        print(score)

    return score

## Evaluation function

In [18]:
def eval_board(board, player, eval_type, to_print=False):
    if four_in_a_row(board, 1):
        # Alice won
        return 100000000
    elif four_in_a_row(board, -1):
        # Bob won
        return -100000000
    else:
        # Not terminal, let's simulate...
        if eval_type==0:
            return montecarlo_e(board, player)
        else:
            return score_position(board, player, to_print)

## Minimax

In [7]:
def minimax(board, depth, alpha, beta, player, maximixingPlayer, eval_type, to_print = False):
    if four_in_a_row(board, 1):
        # Alice won
        return 100000000*maximixingPlayer, None
    elif four_in_a_row(board, -1):
        # Bob won
        return -100000000*maximixingPlayer,  None
    if depth == 0:
        if to_print:
            print('---------------------------')
        # player has to be inverted because we want to evaluate the position from the perspective of the player at depth level before
        static_eval = eval_board(board, -1*player, eval_type, to_print)
        if to_print:
            print(np.rot90(board.copy()))
            print(static_eval)
        # print('*******************************************************')
        # print("max depth status(eval, player): ", static_eval, player)
        return static_eval,  None

    if player == maximixingPlayer:
        max_eval = -float('inf')
        best_move = None
        for column in valid_moves(board):
            n_board = board.copy()
            play(n_board, column, player)
            if column == 3 and depth == 5 or to_print == True:
                eval, _ = minimax(n_board, depth - 1, alpha, beta, -player, maximixingPlayer, eval_type, False)
            else:
                eval, _ = minimax(n_board, depth - 1, alpha, beta, -player, maximixingPlayer, eval_type)
            if eval > max_eval:
                max_eval = eval
                best_move = column
            max_eval = max(max_eval, eval)
            alpha = max(alpha, eval)
            if beta <= alpha:
                break
        return max_eval, best_move
    else:
        min_eval = float('inf')
        for column in valid_moves(board):
            n_board = board.copy()
            play(n_board, column, player)
            if to_print == True:
                eval, _ = minimax(n_board, depth - 1, alpha, beta, -player, maximixingPlayer, eval_type, False)
            else:
                eval, _ = minimax(n_board, depth - 1, alpha, beta, -player, maximixingPlayer, eval_type)
            min_eval = min(min_eval, eval)
            beta = min(beta, eval)
            if beta <= alpha:
                break
        return min_eval, None

## Minimax with heuristic evaluation vs Minimax with montecarlo evaluation (the first starting)
the depth of the minimax with montecarlo evaluation has to be smaller, however, a depth of 2 perform almost as good as a depth of 5 in the other minimax variation and takes more or less the same time.
Unfortunately, sometimes, given the short depth, it is not able to see very trivial traps and it falls for them (this could lead to a lost in very few moves)

In [20]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
minimax_depth = 5
minimax_with_montecarlo_depth = 2
montecarlo_tree_iterations = 1000
computerVcomputer = True
not_finished = True
print(np.rot90(board.copy()))
print('  ')
while not_finished:
    if computerVcomputer == False:
        col = input("column number: ")
        play(board, int(col), 1)
        print('you played: ', col)
        print(np.rot90(board.copy()))
    else:
        t = time.process_time()
        eval, col = minimax(board, minimax_depth, -float('inf'), float('inf'), 1, 1, 1)
        elapsed_time = time.process_time() - t
        play(board, col, 1)
        print('computer 1 (heuristic evaluation) played: ', col, ' evaluate its own score: ', eval)
        print(np.rot90(board.copy()))
        print('elapsed time: ', elapsed_time)
        print('   ')

    if four_in_a_row(board, 1):
        print("PLAYER 1 WON")
        break


    t = time.process_time()
    eval, col = minimax(board, minimax_with_montecarlo_depth, -float('inf'), float('inf'), -1, -1, 0)
    elapsed_time = time.process_time() - t
    play(board, col, -1)
    print('computer -1 (montecarlo evaluation) played: ', col, ' evaluate its own score: ', eval)
    print(np.rot90(board.copy()))
    print('elapsed time: ', elapsed_time)
    print('   ')

    if four_in_a_row(board, -1):
        print("PLAYER -1 WON")
        break


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
  
computer 1 (heuristic evaluation) played:  3  evaluate its own score:  12
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]
elapsed time:  6.859375
   
computer -1 (montecarlo evaluation) played:  0  evaluate its own score:  0.46
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [-1  0  0  1  0  0  0]]
elapsed time:  14.21875
   
computer 1 (heuristic evaluation) played:  3  evaluate its own score:  20
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [-1  0  0  1  0  0  0]]
elapsed time:  8.71875
   
computer -1 (montecarlo evaluation) played:  1  evaluate its own score:  0.56
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  

## Minimax with heuristic evaluation vs Minimax with montecarlo evaluation (the latter starting)
running a bunch of simulations, I've noticed that it's difficult to determine which AI performs better, because the one starting has a great advantage and almost always end up winning. However sometimes the heuristic evaluation minimax is able to win eventhough it starts as second player, being also slightly faster, it results to be the best algorithm

In [9]:

board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
minimax_depth = 5
minimax_with_montecarlo_depth = 2
montecarlo_tree_iterations = 1000
computerVcomputer = True
not_finished = True
print(np.rot90(board.copy()))
print('  ')
while not_finished:
    if computerVcomputer == False:
        col = input("column number: ")
        play(board, int(col), 1)
        print('you played: ', col)
        print(np.rot90(board.copy()))
    else:
        t = time.process_time()
        eval, col = minimax(board, minimax_with_montecarlo_depth, -float('inf'), float('inf'), 1, 1, 0)
        elapsed_time = time.process_time() - t
        play(board, col, 1)
        print('computer 1 (montecarlo evaluation) played: ', col, ' evaluate its own score: ', eval)
        print(np.rot90(board.copy()))
        print('elapsed time: ', elapsed_time)
        print('   ')

    if four_in_a_row(board, 1):
        print("PLAYER 1 WON")
        break


    t = time.process_time()
    eval, col = minimax(board, minimax_depth, -float('inf'), float('inf'), -1, -1, 1)
    elapsed_time = time.process_time() - t
    play(board, col, -1)
    print('computer -1 (heuristic evaluation) played: ', col, ' evaluate its own score: ', eval)
    print(np.rot90(board.copy()))
    print('elapsed time: ', elapsed_time)
    print('   ')

    if four_in_a_row(board, -1):
        print("PLAYER -1 WON")
        break


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
  
computer 1 (montecarlo evaluation) played:  3  evaluate its own score:  -0.08
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]
elapsed time:  29.71875
   
computer -1 (heuristic evaluation) played:  3  evaluate its own score:  9
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]]
elapsed time:  7.25
   
computer 1 (montecarlo evaluation) played:  3  evaluate its own score:  -0.04
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]]
elapsed time:  29.15625
   
computer -1 (heuristic evaluation) played:  2  evaluate its own score:  14
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0]
 [ 0  0  0 -1  0  0 

## Montecarlo tree search against minimax with heuristic evaluation
Lastly I implemented an AI with montecarlo tree search algorithm. Tuning the iterations in order to make it take the almost the same time as minimax I noticed a worst performance (sometimes it lost eventhough it started).
However, it's interesting to see that it starts in the center, eventhough I haven't implemented any heuristic based mechanism

In [14]:

board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
minimax_depth = 5
minimax_with_montecarlo_depth = 2
montecarlo_tree_iterations = 1000
computerVcomputer = True
not_finished = True
print(np.rot90(board.copy()))
print('  ')
while not_finished:
    t = time.process_time()
    montecarlo = MCTS(Model(board.copy()), 1) 
    move = montecarlo.run_search(montecarlo_tree_iterations)
    elapsed_time = time.process_time() - t
    play(board, move, 1)
    print('computer 1 played: ', move)
    print(np.rot90(board.copy()))
    print('elapsed time: ', elapsed_time)
    print('   ')

    if four_in_a_row(board, 1):
        print("PLAYER 1 WON")
        break

    if computerVcomputer == False:
        col = input("column number: ")
        play(board, int(col), -1)
        print('you played: ', col)
        print(np.rot90(board.copy()))
    else:
        t = time.process_time()
        eval, col = minimax(board, minimax_depth, -float('inf'), float('inf'), -1, -1, 1)
        elapsed_time = time.process_time() - t
        play(board, col, -1)
        print('computer -1 played: ', col, ' evaluate its own score: ', eval)
        print(np.rot90(board.copy()))
        print('elapsed time: ', elapsed_time)
        print('   ')

    if four_in_a_row(board, -1):
        print("PLAYER -1 WON")
        break

[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
  
computer 1 played:  3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]]
elapsed time:  19.828125
   
computer -1 played:  3  evaluate its own score:  9
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0]]
elapsed time:  7.5
   
computer 1 played:  2
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  1  1  0  0  0]]
elapsed time:  15.5
   
computer -1 played:  4  evaluate its own score:  10
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0]
 [ 0  0  1  1 -1  0  0]]
elapsed time:  4.828125
   
computer 1 played:  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0 

## MCTS test
just a test to monitor if the AI choose the best move when there's only very few of them

In [13]:
board = board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
play(board, 1, 1)
play(board, 6, -1)
play(board, 2, 1)
play(board, 6, -1)
minimax_depth = 5
minimax_with_montecarlo_depth = 2
montecarlo_tree_iterations = 2000
computerVcomputer = True
not_finished = True
print(np.rot90(board.copy()))
print('  ')
while not_finished:
    t = time.process_time()
    montecarlo = MCTS(Model(board.copy()), 1) 
    move = montecarlo.run_search(montecarlo_tree_iterations)
    elapsed_time = time.process_time() - t
    play(board, move, 1)
    print('computer 1 played: ', move)
    print(np.rot90(board.copy()))
    print('elapsed time: ', elapsed_time)
    print('   ')

    if four_in_a_row(board, 1):
        print("PLAYER 1 WON")
        break

    if computerVcomputer == False:
        col = input("column number: ")
        play(board, int(col), -1)
        print('you played: ', col)
        print(np.rot90(board.copy()))
    else:
        t = time.process_time()
        eval, col = minimax(board, minimax_depth, -float('inf'), float('inf'), -1, -1, 1)
        elapsed_time = time.process_time() - t
        play(board, col, -1)
        print('computer -1 played: ', col, ' evaluate its own score: ', eval)
        print(np.rot90(board.copy()))
        print('elapsed time: ', elapsed_time)
        print('   ')

    if four_in_a_row(board, -1):
        print("PLAYER -1 WON")
        break

[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 0  1  1  0  0  0 -1]]
  
computer 1 played:  3
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [ 0  1  1  1  0  0 -1]]
elapsed time:  10.703125
   
computer -1 played:  0  evaluate its own score:  -100000000
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [-1  1  1  1  0  0 -1]]
elapsed time:  1.46875
   
computer 1 played:  4
[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1]
 [-1  1  1  1  1  0 -1]]
elapsed time:  4.0
   
PLAYER 1 WON


## Conclusion
at the end the minimax with heuristic evaluation seems to be the best algorithm, both in term of speed and quality of the decisions. The fact that the evaluation is faster, permits to go deeper in the decision tree (the heuristic behind the evaluation could even be improved)